In [2]:
import os
%pwd

'd:\\End to End NLP\\research'

In [3]:
os.chdir('..')
%pwd

'd:\\End to End NLP'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([Path(config.root_dir)])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [10]:
import os
from src.textSummarizer.logger import logging
from transformers import AutoTokenizer
from datasets import load_from_disk

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        logging.info("Converting examples to features...")
        input_embeddings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation = True)

        logging.info("Converting summaries to features...")
        with self.tokenizer.as_target_tokenizer():
            target_embeddings = self.tokenizer(example_batch['summary'], max_length=128, truncation = True)

        return {'input_ids': input_embeddings['input_ids'],
                'attention_mask': input_embeddings['attention_mask'],
                'labels': target_embeddings['input_ids']}

    def convert(self):
        logging.info("Loading dataset from disk...")
        dataset_samsum = load_from_disk(self.config.data_path)

        logging.info("Transforming dataset...")
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)

        logging.info("Saving transformed dataset to disk...")
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))


In [15]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2025-11-16 21:34:32,447] 15 - INFO - YAML file: config\config.yaml loaded successfully
[2025-11-16 21:34:32,448] 15 - INFO - YAML file: params.yaml loaded successfully
[2025-11-16 21:34:32,448] 27 - INFO - Directory created at: artifacts
[2025-11-16 21:34:32,451] 27 - INFO - Directory created at: artifacts\data_transformation
[2025-11-16 21:34:34,120] 19 - INFO - Loading dataset from disk...
[2025-11-16 21:34:34,250] 22 - INFO - Transforming dataset...


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

[2025-11-16 21:34:34,283] 7 - INFO - Converting examples to features...
[2025-11-16 21:34:34,373] 10 - INFO - Converting summaries to features...


d:\End to End NLP\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map:   7%|▋         | 1000/14732 [00:00<00:02, 5625.17 examples/s]

[2025-11-16 21:34:34,461] 7 - INFO - Converting examples to features...
[2025-11-16 21:34:34,542] 10 - INFO - Converting summaries to features...


...

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

[2025-11-16 21:34:36,746] 7 - INFO - Converting examples to features...
[2025-11-16 21:34:36,806] 10 - INFO - Converting summaries to features...


Map: 100%|██████████| 818/818 [00:00<00:00, 6276.03 examples/s]

[2025-11-16 21:34:36,876] 25 - INFO - Saving transformed dataset to disk...



Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 108622.20 examples/s]
